In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3

In [3]:
loc_df = pd.read_csv('META_관측지점정보_20200807155525.csv')
loc_df.head()

,지점,시작일,종료일,지점명,지점주소,관리관서,위도,경도,노장해발고도(m),기압계(관측장비지상높이(m)),기온계(관측장비지상높이(m)),풍속계(관측장비지상높이(m)),강우계(관측장비지상높이(m))
0,105,1911-10-03,NaN,강릉,강원도 강릉시용강동 63-20강릉자동기상관측소,강릉(구 105),37.7515,128.8910,27.12,28.22,1.7,10.0,0.5
1,259,2009-11-10,NaN,강진군,전라남도 강진군강진읍 금릉1길48 전남수산기술사업소 강진지소,목포기상대(165),34.6261,126.7689,12.50,13.90,1.5,10.0,0.6
2,201,1972-01-11,NaN,강화,인천광역시 강화군불은면 삼성리811-1 강화기상관측소,인천기상대(112),37.7074,126.4463,47.84,49.24,1.6,10.0,0.6
3,294,1972-01-24,NaN,거제,경상남도 거제시장평2로 2길47 거제기상관측소,부산지방기상청(159),34.8882,128.6045,45.40,46.70,1.7,10.0,0.6
4,284,2011-03-04,NaN,거창,경상남도 거창군거창읍 수남로2117 거창지역기상서비스센터,울산기상대(152),35.6674,127.9090,225.95,227.30,1.5,10.0,1.3


In [4]:
df = loc_df[['지점', '시작일', '종료일', '위도', '경도']]

In [5]:
df.head()

,지점,시작일,종료일,위도,경도
0,105,1911-10-03,NaN,37.7515,128.8910
1,259,2009-11-10,NaN,34.6261,126.7689
2,201,1972-01-11,NaN,37.7074,126.4463
3,294,1972-01-24,NaN,34.8882,128.6045
4,284,2011-03-04,NaN,35.6674,127.9090


In [8]:
len(df)

135

In [11]:
df = df.drop_duplicates(subset=['지점'])
df.head()

,지점,시작일,종료일,위도,경도
0,105,1911-10-03,NaN,37.7515,128.8910
1,259,2009-11-10,NaN,34.6261,126.7689
2,201,1972-01-11,NaN,37.7074,126.4463
3,294,1972-01-24,NaN,34.8882,128.6045
4,284,2011-03-04,NaN,35.6674,127.9090


In [14]:
# Extract 지점, 위도, 경도
loc_values = df[['지점', '위도','경도']].values
loc_values[:5]

array([[105.    ,  37.7515, 128.891 ],
       [259.    ,  34.6261, 126.7689],
       [201.    ,  37.7074, 126.4463],
       [294.    ,  34.8882, 128.6045],
       [284.    ,  35.6674, 127.909 ]])

In [23]:
# DB 연결 및 저장경로 설정
save_dir='.'
conn = sqlite3.connect(save_dir + '/wdb.db', isolation_level=None)  # auto-commit
c = conn.cursor()

# 테이블 위도, 경도 컬럼 추가
c.execute("ALTER TABLE cities ADD COLUMN lat REAL DEFAULT 0")
c.execute("ALTER TABLE cities ADD COLUMN long REAL DEFAULT 0")

conn.close()

In [29]:
# 지점(city_id)에 맞게 위도, 경도 데이터 집어넣기
save_dir='.'
conn = sqlite3.connect(save_dir + '/wdb.db', isolation_level=None)  # auto-commit
c = conn.cursor()

for row in loc_values:
    # search and insert
    c.execute(f'UPDATE cities SET lat={row[1]}, long={row[2]} WHERE city_id={int(row[0])}')
conn.close()